In [1]:
import os

In [2]:
%pwd

'd:\\New Projects\\Customer Churn Prediction - End to End (ML)\\Customer-Churn-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\New Projects\\Customer Churn Prediction - End to End (ML)\\Customer-Churn-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [6]:
from CustomerChurn.constants import *
from CustomerChurn.utils.common import read_yaml, create_directories, save_json

In [11]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.XGBClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [8]:
import pandas as pd 
from sklearn.metrics import accuracy_score, f1_score, recall_score
import numpy as np 
import joblib

In [16]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self,actual, pred):
        accuracy = accuracy_score(actual, pred)
        f1_Score = f1_score(actual, pred)
        recall_Score = recall_score(actual, pred)
        return accuracy, f1_Score, recall_Score
    
    def save_results(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        X_test = test_data.drop([self.config.target_column], axis=1)
        y_test = test_data[[self.config.target_column]]
        
        predicted = model.predict(X_test)

        (accuracy, f1_Score, recall_Score) = self.eval_metrics(y_test, predicted)
        
        # Saving metrics as local
        scores = {"accuracy": accuracy, "f1_Score": f1_Score, "recall_Score": recall_Score}
        save_json(path=Path(self.config.metric_file_name), data=scores)

In [17]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.save_results()
except Exception as e:
    raise e

[2024-09-20 14:40:51,268] 32 CustomerChurnLogger - common - INFO - yaml file: config\config.yaml loaded successfully
[2024-09-20 14:40:51,276] 32 CustomerChurnLogger - common - INFO - yaml file: params.yaml loaded successfully
[2024-09-20 14:40:51,281] 32 CustomerChurnLogger - common - INFO - yaml file: schema.yaml loaded successfully
[2024-09-20 14:40:51,283] 52 CustomerChurnLogger - common - INFO - Created directory at: artifacts
[2024-09-20 14:40:51,284] 52 CustomerChurnLogger - common - INFO - Created directory at: artifacts/model_evaluation
[2024-09-20 14:40:51,384] 68 CustomerChurnLogger - common - INFO - json file saved at: artifacts\model_evaluation\metrics.json
